In [13]:
import pandas as pd
import numpy as np
import time

# Data Cleaning

In [3]:
def date_to_int(row, col, df):
    index = df.columns.get_loc(col)
    date = row[index]
    return time.mktime(time.strptime(str(date), '%Y-%m-%d %H:%M:%S'))

In [4]:
def data_type(df):
    # datetime and date to int
    date_cols = ['visit_date', 'prev_visit_date', 'prev_item_move_date', 
                 'last_edit_date', 'creation_date']
    for col in date_cols:
        # convert multiple time formats into single string format
        df[col] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d %H:%M:%S')
        # make time features specific data type in order to distinguish from other numberic values
        df['{}_int'.format(col)] = df.apply(date_to_int, col=col, df=df, axis=1).astype(np.float32)
        # convert string format back into datetime
        df[col] = pd.to_datetime(df[col])
    
    # objects
    obj_cols = ['ship_id', 'address1', 'customer_id', 'sales_rep_id', 'item_id', 'old_item_id', 
                'item_UPC', 'old_item_UPC', 'ship_list_pk', 'sales_rep_id_2', 'list_header_id']
    for col in obj_cols:
        df[col] = df[col].astype(object)

In [5]:
def nans(df):
    df.dropna(axis=0, inplace=True)
    # caution: crime data cuts total data in ~1/3

In [6]:
def zip_code_inc(row, df):
    index = df.columns.get_loc('postal_code')
    code = row[index]
    return int(code[:5])

In [7]:
def days_between_visits(df):
    out_arr = []
    i = 0
    for index, row in df.iterrows():
        diff = row['visit_date'] - row['prev_visit_date']
        out_arr.append(pd.Timedelta(diff).days)
    return out_arr

In [ ]:
# not working
def get_masked_df(df, level_list):
    for ff_val in df[first_filter].unique():
        ff_mask = df[first_filter] == ff_val
        for sf_val in df[ff_mask][second_filter].unique():
            sf_mask = df[second_filter] == sf_val
            for tf_val in df[ ff_mask & sf_mask][third_filter].unique():
                tf_mask = df[third_filter] == tf_val
                foo = df[ ff_mask & sf_mask & tf_mask ].sort_values('visit_date', ascending=False)
    return df[ mask ].sort_values('visit_date', ascending=False)

In [ ]:
# not working
def lag3(df, num_periods=1, first_filter='address1', second_filter='item_category', third_filter='item_upc'):
    # initialize column(s) of nans
    for period in range(1, num_periods + 1):
        if period == 1:
            continue
        df['qty_shrink_per_day_lag{}'.format(period)] = np.nan
        df['shrink_value_per_day_lag{}'.format(period)] = np.nan
        
    j = 0
    for ff_val in df[first_filter].unique():
        j += 1
        if j % 1000 == 0:
            print('another 10%...')
        ff_mask = df[first_filter] == ff_val
        for sf_val in df[ff_mask][second_filter].unique():
            sf_mask = df[second_filter] == sf_val
            for tf_val in df[ ff_mask & sf_mask][third_filter].unique():
                tf_mask = df[third_filter] == tf_val
                foo = df[ ff_mask & sf_mask & tf_mask ].sort_values('visit_date', ascending=False)
                #print('foo: ',foo[['visit_date']])
                length = len(foo.visit_date.unique()) # determine number of visits (because multiple item categories can be updated in a single visit)
                for period in range(1, num_periods + 1):
                    if period == 1:
                        continue
                    # skip if there's not enough data to create lag variables
                    filt_list = [first_filter, second_filter, third_filter]
                    end_of_filters = 0
                    while (length < period + 1) | end_of_filters:
                        foo = get_masked_df(df, filt_list)
                        # length = len(foo.visit_date.unique())
                        filt_list.pop()
                        continue
                    #print('length: ',length)
                    i = 0
                    foo_shifted = foo.shift(-period)
                    #print('fs: ', foo_shifted[['visit_date']])
                    foo_grouped = foo.groupby('visit_date').mean()
                    #print('fg: ', foo_shifted[['visit_date']])
                    for index, row in foo.iterrows():
                        #print(index)
                        date = foo_shifted[ foo_shifted.index == index].visit_date.values[0]
                        qty = foo_grouped[ foo_grouped.index == date].qty_shrink_per_day.values[0]
                        value = foo_grouped[ foo_grouped.index == date].shrink_value_per_day.values[0]
                        df.set_value(index, 'qty_shrink_per_day_lag{}'.format(period), qty)
                        df.set_value(index, 'shrink_value_per_day_lag{}'.format(period), value)
                        i += 1
                        #print(i)
                        if i + period == length:
                            break # back to cat loop

In [8]:
def lag(df, num_periods=1, first_filter='address1', second_filter='item_category', date_filter='visit_date',
        lag_var1='qty_shrink_per_day', lag_var2='shrink_value_per_day', col_name_suf=''):
    
    # initialize column(s) of nans
    for period in range(1, num_periods + 1):
        df['{}_lag{}{}'.format(lag_var1, period, col_name_suf)] = np.nan
        df['{}_lag{}{}'.format(lag_var2, period, col_name_suf)] = np.nan
        
    for ff_val in df[first_filter].unique():
        ff_mask = df[first_filter] == ff_val
        for sf_val in df[ff_mask][second_filter].unique():
            sf_mask = df[second_filter] == sf_val
            foo = df[ ff_mask & sf_mask ].sort_values(date_filter, ascending=False)
            #print('foo: ',foo[['visit_date']])
            length = len(foo[date_filter].unique()) # determine number of visits (because multiple item categories can be updated in a single visit)
            for period in range(1, num_periods + 1):
                # skip if there's not enough data to create lag variables
                if length < period + 1:
                    continue
                i = 0
                foo_shifted = foo.shift(-period)
                foo_grouped = foo.groupby(date_filter).mean()
                for index, row in foo.iterrows():
                    date = foo_shifted[ foo_shifted.index == index][date_filter].values[0]
                    lag1_val = foo_grouped[ foo_grouped.index == date][lag_var1].values[0]
                    lag2_val = foo_grouped[ foo_grouped.index == date][lag_var2].values[0]
                    
                    # set values
                    df.set_value(index, '{}_lag{}{}'.format(lag_var1, period, col_name_suf), lag1_val)
                    df.set_value(index, '{}_lag{}{}'.format(lag_var2, period, col_name_suf), lag2_val)
                    i += 1
                    if i + period == length:
                        break # back to cat loop

In [ ]:
def lag_old(df, num_periods=1):
    df['qty_shrink_per_day_lag1'] = np.nan
    df['shrink_value_per_day_lag1'] = np.nan
    j = 0
    for add in df.address1.unique():
        j += 1
        if j % 1000 == 0:
            print('another 10%...')
        add_mask = df.address1 == add
        for cat in df[add_mask].item_category.unique():
            cat_mask = df.item_category == cat
            foo = df[ add_mask & cat_mask].sort_values('visit_date', ascending=False)
            #print('foo: ',foo[['visit_date']])
            length = len(foo.visit_date.unique()) # determine number of visits (because multiple item categories can be updated in a single visit)
            # skip if there's not enough data to create lag variables
            if length < num_periods + 1:
                continue
            #print(length)
            i = 1
            foo_shifted = foo.shift(-num_periods)
            #print('fs: ', foo_shifted[['visit_date']])
            foo_grouped = foo.groupby('visit_date').mean()
            for index, row in foo.iterrows():
                #print(index)
                date = foo_shifted[ foo_shifted.index == index].visit_date.values[0]
                qty = foo_grouped[ foo_grouped.index == date].qty_shrink_per_day.values[0]
                value = foo_grouped[ foo_grouped.index == date].shrink_value_per_day.values[0]
                df.set_value(index, 'qty_shrink_per_day_lag1', qty)
                df.set_value(index, 'shrink_value_per_day_lag1', value)
                i += 1
                #print(i)
                if i == length:
                    break # back to cat loop

In [9]:
def create(df, lag_periods):
    df['zip_code'] = df.apply(zip_code_inc, df=df, axis=1)
    
    # normalize target variables
    days_list = days_between_visits(df)
    df['qty_shrink_per_day'] = df.qty_shrink / days_list
    df['shrink_value_per_day'] = df.shrink_value / days_list
    
    # add lag variables
    lag(df, num_periods=lag_periods) # caution: takes a long time
    return df

In [10]:
def drop(df):
    del df['address3'] # redundant info (same as address 2)
    del df['postal_code'] # create zip code
    del df['duration'] # all zero values
    del df['dist_customer_id'] # all -1 values
    del df['POG_version_timestamp'] # dup of visit_date

In [11]:
def dummy(df):
    dummy_cols = ['item_category', 'customer_id']
    foo = pd.DataFrame()
    foo[dummy_cols] = df[dummy_cols].astype(str)
    df = pd.get_dummies(df, columns=dummy_cols)
    df[dummy_cols] = foo[dummy_cols]
    del foo
    return df

In [14]:
def clean(file, lag_periods, remove_nan_rows=True):
    df = pd.read_pickle(file)
    data_type(df)
    df = create(df, lag_periods)
    drop(df)
    df = dummy(df)
    if remove_nan_rows:
        nans(df)
    return df

In [15]:
# get SRP data and clean
df = clean(file='data/SRP/raw_subset_300k.pkl', lag_periods=2, remove_nan_rows=True)
# lag of 2 seems to be best bet

/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [17]:
df.to_pickle('data/SRP/clean_data_no_public.pkl')

# Add public data

In [26]:
df = pd.read_pickle('data/SRP/clean_data_no_public.pkl')

In [20]:
def add_data(df, include_crime=True, remove_nan_rows=True):
    # load data
    fd = pd.read_pickle('data/Food_Deserts/FD_clean.pkl').set_index('Zip Code')
    unemp = pd.read_pickle('data/Unemployment/unemp_clean.pkl').set_index('Zip')
    #inc = pd.read_pickle('data/Income/income_clean.pkl').set_index('ZIPCODE')
    dens = pd.read_pickle('data/Pop_Density/density_clean.pkl').set_index('Zip/ZCTA')
    crime = pd.read_pickle('data/Crime/grouped_clean.pkl').set_index(['state', 'city'])

    # join via zip code
    df = df.join(fd, on=['zip_code'], how='left')
    df = df.join(unemp, on=['zip_code'], how='left')
    # df = df.join(inc, on=['zip_code'], how='left')
    df = df.join(dens, on=['zip_code'], how='left')
    df['dens_sq_mile'] = df['dens/sq_mile'].replace(0, np.nan)
    del df['dens/sq_mile']
    
    # join via city/state
    if include_crime:
        df = df.join(crime, on=['state', 'city'], how='left')
        
    # drop all rows that contain nan
    if remove_nan_rows:
        nans(df)
        
    return df

In [21]:
def zip_code_str(row, df):
    index = df.columns.get_loc('zip_code')
    code = row[index]
    return str(code).zfill(5)

In [27]:
df = add_data(df, include_crime=True, remove_nan_rows=True)
df['zip_code'] = df.apply(zip_code_str, df=df, axis=1)

# still need to impute nans and 0's

In [29]:
df.to_pickle('data/SRP/clean_data_public_all.pkl')

# POA
- Create averages:
    - Avg qty shrink/day, shink_sales/day, etc
- Engineer lag terms (ie last visit, last month, last season)
    - Use these in whatever model I want
    - Use the averaged values